In [1]:
from v3_pool import *

In [2]:
Token("ETH",18)

Token(name='ETH', decimals=18)

In [3]:
GlobalState()

GlobalState(L=0.0, rP=0.0, tick=0, fg_x=0.0, fg_y=0.0, proto_x=0.0, proto_y=0.0)

In [4]:
TickState()

TickState(liq_net=0.0, liq_gross=0.0, f0_x=0.0, f0_y=0.0)

In [5]:
PositionState()

PositionState(liq=0.0, fr_x=0.0, fr_y=0.0)

In [6]:
pool=Pool("ETH", 18, "USDC", 6)
pool

ETH-USDC pool
Token(name='ETH', decimals=18) Token(name='USDC', decimals=6)
GlobalState(L=0.0, rP=0.0, tick=0, fg_x=0.0, fg_y=0.0, proto_x=0.0, proto_y=0.0)


Problem 1:  
A user has x = 2 ETH and wants to set up a liquidity position in an ETH/USDC pool. The current price of ETH is P =2000 USDC and target price range is from pa = 1500 to pb = 2500 USDC. How much USDC (y) do they need?

In [7]:
# 2-step: get L first
a,P,b = 1500,2000,2500
x = 2.0 
# First, calculate the liquidity of the top half of the range by using
# reserve x cover from current P to upper bound (pretend P is lower bound)
Lx = Pool.liq_x_only(x=x,rPa=P**0.5, rPb=b**0.5)
Lx

847.2135954999583

In [8]:
# 2-step: then get y . 
y = Pool.y_from_L_rP_rng(L=Lx,rP=P**0.5,rPa=a**0.5,rPb=b**0.5)
y

5076.102359479882

Problem 2:  
A user has x=2 ETH and y=4000 USDC, and wants to use pb =3000USDC perETH as the top of the price range. What is the bottom of the range (pa) that ensures the opened position uses the full amount of their funds?

In [9]:
# Bounds in 1-step
x,y  = 2.0, 4000
P = y/x
b= 3000

rPa = Pool.rPa_from_x_y_rP_rPb(x=x,y=y,rP=P**0.5,rPb=b**0.5)
rPa**2

1333.3333333333333

In [10]:
# Bounds in 2-steps
# compute L using the top part of range, given we know x 
Lx = Pool.liq_x_only(x,rPa=P**0.5,rPb=b**0.5)
# then get lower bound from L, price and y
rPa = Pool.rPa_from_L_rP_y(L=Lx,rP=P**0.5,y=y)
rPa**2

1333.3333333333337

Problem 3:  
Using the liquidity position created in Problem 2, what are asset balances when the price changes to P = 2500 USDC per ETH?

In [11]:
# compute L from price when position established, reserves and bounds
L = Pool.liq_from_x_y_rP_rng(x=x,y=y,rP=P**0.5, rPa=rPa, rPb=b**0.5)

x_new = Pool.x_from_L_rP_rng(L=L, rP=2500**0.5, rPa=rPa, rPb=b**0.5)

y_new = Pool.y_from_L_rP_rng(L=L, rP=2500**0.5, rPa=rPa, rPb=b**0.5)

x_new , y_new

(0.8493641204744684, 6572.9000439693455)

In [12]:
# or calc change of reserves based on change of price
dX =  Pool.dX_from_L_drP(L,rP_old=P**0.5,rP_new=2500**0.5)
print(f"{L=} {dX=} x_new={x + dX}")

L=487.41718030204123 dX=-1.1506358795255318 x_new=0.8493641204744682


In [13]:
dY =  Pool.dY_from_L_drP(L,rP_old=P**0.5,rP_new=2500**0.5)
print(f"{dY=} y_new={y + dY}")

dY=2572.9000439693473 y_new=6572.900043969347


In [14]:
pool.set_position("123", -10, 30, 300)
pool.show()

GlobalState(L=300.0, rP=0.0, tick=0, fg_x=0.0, fg_y=0.0, proto_x=0.0, proto_y=0.0)

---active ticks---
tick '-10': TickState(liq_net=300.0, liq_gross=300.0, f0_x=0.0, f0_y=0.0)
tick '30': TickState(liq_net=-300.0, liq_gross=300.0, f0_x=0, f0_y=0)
---positions---
poz '('123', -10, 30)': PositionState(liq=300, fr_x=0.0, fr_y=0.0)


In [15]:
pool.set_position("888", 30, 50, 200)
pool.show()

GlobalState(L=300.0, rP=0.0, tick=0, fg_x=0.0, fg_y=0.0, proto_x=0.0, proto_y=0.0)

---active ticks---
tick '-10': TickState(liq_net=300.0, liq_gross=300.0, f0_x=0.0, f0_y=0.0)
tick '30': TickState(liq_net=-100.0, liq_gross=500.0, f0_x=0, f0_y=0)
tick '50': TickState(liq_net=-200.0, liq_gross=200.0, f0_x=0, f0_y=0)
---positions---
poz '('123', -10, 30)': PositionState(liq=300, fr_x=0.0, fr_y=0.0)
poz '('888', 30, 50)': PositionState(liq=200, fr_x=0.0, fr_y=0.0)


In [16]:
pool.set_position("888", 30, 50, -200)
pool.show()

GlobalState(L=300.0, rP=0.0, tick=0, fg_x=0.0, fg_y=0.0, proto_x=0.0, proto_y=0.0)

---active ticks---
tick '-10': TickState(liq_net=300.0, liq_gross=300.0, f0_x=0.0, f0_y=0.0)
tick '30': TickState(liq_net=-300.0, liq_gross=300.0, f0_x=0, f0_y=0)
---positions---
poz '('123', -10, 30)': PositionState(liq=300, fr_x=0.0, fr_y=0.0)
